<a href="https://www.kaggle.com/code/nadirfartas/pfenotebook?scriptVersionId=236321355" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

secret_label = "HF Hub"
secret_value = UserSecretsClient().get_secret(secret_label)
login(token=secret_value)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [20]:
!pip install \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    evaluate \
    accelerate \
    bitsandbytes \
    trl \
    peft \
    -U --quiet

In [27]:
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.8 MB/s eta 0:00:00


In [28]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import pandas as pd
import re
import numpy as np
import string
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
import evaluate

In [29]:
%pip install -q -U git+https://github.com/huggingface/peft.git

Note: you may need to restart the kernel to use updated packages.


In [30]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [31]:
import transformers

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model = AutoModelForSeq2SeqLM.from_pretrained("UBC-NLP/araT5-base")
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/araT5-base")

ValueError: Converting from SentencePiece and Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast converters: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'Realm

In [21]:
# Set pad_token as end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [22]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 282770688
all model parameters: 282770688
percentage of trainable model parameters: 100.00%


In [23]:
from datasets import load_dataset

ds = load_dataset("ZeyadAhmed/Arabic-SQuADv2.0")

asquadv2-train.json:   0%|          | 0.00/91.9M [00:00<?, ?B/s]

asquadv2-val.json:   0%|          | 0.00/27.7M [00:00<?, ?B/s]

asquadv2-test.json:   0%|          | 0.00/27.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [10]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['version', 'data'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['version', 'data'],
        num_rows: 1
    })
    test: Dataset({
        features: ['version', 'data'],
        num_rows: 1
    })
})


In [25]:
def extract_grouped_examples(dataset_split):
    examples = []
    for article in dataset_split[0]['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context'].strip()
            questions = []
            for qa in paragraph['qas']:
                question = qa['question'].strip()
                if question:
                    questions.append(question)

            if context and questions:
                # Join all questions into one string separated by " <sep> "
                target = " <sep> ".join(questions)
                examples.append({
                    "input": context,
                    "target": target
                })
    return examples

# Apply the function to both splits
train_examples = extract_grouped_examples(ds["train"])
val_examples = extract_grouped_examples(ds["validation"])


In [26]:
print(train_examples[0])

{'input': '( لم يكن زلزال Ms 6 . 1 في 30 أغسطس 2008 في جنوب سيتشوان جزءا من هذه السلسلة لأنه كان ناجما عن صدع مختلف . انظر زلزال بانتشيهوا 2008 للحصول على التفاصيل . )', 'target': 'أين كان زلزال 30 أغسطس 2008 ؟ <sep> أين يجب أن تبحث عن مزيد من التفاصيل ؟ <sep> متى وقع هذا الزلزال ؟'}


In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

CUDA available: False
Device: cpu


NameError: name 'model' is not defined

In [28]:
print(f"Train samples: {len(train_examples)}")
print(f"Validation samples: {len(val_examples)}")

Train samples: 18117
Validation samples: 7290


In [29]:
print(train_examples[0])
import transformers
print(transformers.__version__)

{'input': '( لم يكن زلزال Ms 6 . 1 في 30 أغسطس 2008 في جنوب سيتشوان جزءا من هذه السلسلة لأنه كان ناجما عن صدع مختلف . انظر زلزال بانتشيهوا 2008 للحصول على التفاصيل . )', 'target': 'أين كان زلزال 30 أغسطس 2008 ؟ <sep> أين يجب أن تبحث عن مزيد من التفاصيل ؟ <sep> متى وقع هذا الزلزال ؟'}
4.51.1


In [30]:
# Add <sep> as a special token
special_tokens_dict = {'additional_special_tokens': ['<sep>']}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(110001, 768)

In [31]:
from datasets import Dataset

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_examples)
val_dataset = Dataset.from_list(val_examples)

# Tokenization function
def tokenize_function(example):
    model_inputs = tokenizer(
        example["input"], 
        max_length=512, 
        padding="max_length", 
        truncation=True
    )
    labels = tokenizer(
        example["target"], 
        max_length=256, 
        padding="max_length", 
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize
tokenized_train = train_dataset.map(tokenize_function, batched=False)
tokenized_val = val_dataset.map(tokenize_function, batched=False)


Map:   0%|          | 0/18117 [00:00<?, ? examples/s]

Map:   0%|          | 0/7290 [00:00<?, ? examples/s]

In [32]:
print(tokenized_train[0])

{'input': '( لم يكن زلزال Ms 6 . 1 في 30 أغسطس 2008 في جنوب سيتشوان جزءا من هذه السلسلة لأنه كان ناجما عن صدع مختلف . انظر زلزال بانتشيهوا 2008 للحصول على التفاصيل . )', 'target': 'أين كان زلزال 30 أغسطس 2008 ؟ <sep> أين يجب أن تبحث عن مزيد من التفاصيل ؟ <sep> متى وقع هذا الزلزال ؟', 'input_ids': [26, 83, 804, 21204, 955, 19, 488, 3, 4, 383, 8, 753, 6964, 1990, 8, 1023, 359, 25784, 131, 5399, 6, 89, 39601, 1340, 77, 70068, 16, 35, 3, 49774, 1099, 3, 4, 10365, 21204, 1471, 13948, 12932, 1990, 5286, 15, 3852, 3, 4, 3, 29, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [71]:
import os
os.environ["WANDB_DISABLED"] = "true"
import torch

# Check if CUDA (GPU) is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model = model.to(device)
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./araT5-multiquestion",         # Where to save model
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,                                   # Use mixed precision if on GPU
    logging_dir="./logs",
    logging_steps=100,
    report_to="none"                             # Disable wandb
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)

# 🚀 Start training
trainer.train()
trainer.save_model("./araT5-multiquestion")
tokenizer.save_pretrained("./araT5-multiquestion", legacy=False)
print("trained well x) ...hopefully!")

Using device: cuda


/tmp/ipykernel_31/3534158778.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: 

Device set to use cuda:0


Q1: ما هي عدد عدد عدد عدد عدد عدد عدد عدد عدد


In [70]:
def get_question(context):
    # Ensure inputs are on the same device as the model
    text_encoding = tokenizer(
        context,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    ).to(model.device)  # Move to the model's device
      model.eval()
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask']
        max_length=64,
        num_beams=5,
        num_return_sequences=1
  )
  return tokenizer.decode(generated_ids[0],skip_special_tokens=True,clean_up_tokenization_spaces=True).replace('question: ',' ')


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

In [66]:
context = "تعد الشمس مصدر الطاقة الرئيسي للأرض، وتلعب دورًا مهمًا في الطقس والمناخ."
get_question(context)



RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)